## Imports

In [1]:
import plotly.graph_objs as go
import numpy as np
import pandas as pd
import yfinance as yf
import functions as func

## Fetching data

### name = name of the stock or cryptocurrency we want to fetch
### start = date to start fetching from
### end = date to finish fetching to

In [2]:
name = "NKLA"
start = "2020-01-01"
#end = "2020-07-07" #znamo da bi tada bot kupio
end = "2020-08-19" #znamo da bi tada bot prodo

In [3]:
data = func.get_data(name, start, end)
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-12-31,10.317000,10.320000,10.315000,10.320000,10.320000,6500
2020-01-02,10.310000,10.320000,10.310000,10.320000,10.320000,60300
2020-01-03,10.300000,10.300000,10.300000,10.300000,10.300000,15000
2020-01-06,10.325000,10.350000,10.320000,10.320000,10.320000,68700
2020-01-07,10.440000,10.440000,10.330000,10.330000,10.330000,42900
...,...,...,...,...,...,...
2020-08-12,42.000000,43.639999,41.410999,42.810001,42.810001,10789500
2020-08-13,43.549999,47.450001,43.310001,45.970001,45.970001,23804300
2020-08-14,46.869999,46.950001,44.360001,45.959999,45.959999,12493900


# Create wallet

In [4]:
#func.create_wallet(name, 10000, "ICHIMOKU CLOUD")

## Plotting the data

In [5]:
func.plot_data(data)

# Ichimoku cloud
#### Documentation: https://www.investopedia.com/terms/i/ichimoku-cloud.asp

### Data manipulation

In [6]:
index = pd.date_range(end, periods=25, freq='D') # datumi narednih 25 dana od zadnjeg dana u podacima
columns = data.columns
data_pred = pd.DataFrame(index=index, columns=columns) #prazna tablica za predikciju sljedecih 25 dana
data = pd.concat([data,data_pred])
data

,Open,High,Low,Close,Adj Close,Volume
2019-12-31,10.317,10.32,10.315,10.32,10.32,6500
2020-01-02,10.310,10.32,10.310,10.32,10.32,60300
2020-01-03,10.300,10.30,10.300,10.30,10.30,15000
2020-01-06,10.325,10.35,10.320,10.32,10.32,68700
2020-01-07,10.440,10.44,10.330,10.33,10.33,42900
...,...,...,...,...,...,...
2020-09-08,NaN,NaN,NaN,NaN,NaN,NaN
2020-09-09,NaN,NaN,NaN,NaN,NaN,NaN
2020-09-10,NaN,NaN,NaN,NaN,NaN,NaN
2020-09-11,NaN,NaN,NaN,NaN,NaN,NaN


## Conversion Line (tenkan sen)
                         
### $CL = \frac{​​(9-PH)+(9-PL)}{2}$

PH - period high
PL - period low
CL - conversion line

In [7]:
data['tenkan_sen'] = (data['High'].rolling(window = 9).max() + data['Low'].rolling(window = 9).min()) / 2

# tek nakon 8 dana dobivamo podatke za tankan sen, tj. 9. dana

data.head(10)

,Open,High,Low,Close,Adj Close,Volume,tenkan_sen
2019-12-31,10.317,10.32,10.315,10.320,10.320,6500,NaN
2020-01-02,10.310,10.32,10.310,10.320,10.320,60300,NaN
2020-01-03,10.300,10.30,10.300,10.300,10.300,15000,NaN
2020-01-06,10.325,10.35,10.320,10.320,10.320,68700,NaN
2020-01-07,10.440,10.44,10.330,10.330,10.330,42900,NaN
2020-01-08,10.330,10.35,10.300,10.342,10.342,1247500,NaN
2020-01-09,10.350,10.35,10.350,10.350,10.350,168700,NaN
2020-01-10,10.350,10.35,10.330,10.330,10.330,370000,NaN
2020-01-13,10.350,10.35,10.340,10.340,10.340,25600,10.37
2020-01-14,10.350,10.35,10.350,10.350,10.350,10000,10.37


## Base Line (kijun sen)

### $BL = \frac{​​(26-PH)+(26-PL)}{2}$

PH - period high
PL - period low
BL - base line

In [8]:
data['kijun_sen'] = (data['High'].rolling(window = 26).max() + data['Low'].rolling(window = 26).min()) / 2

## Leading Span A (senkou span A)

### leading span a = $\frac{CL+BL}{2}$

CL - Conversion Line
BL - Base Line

In [9]:
data['senkou_span_A'] = ((data['tenkan_sen'] + data['kijun_sen']) / 2).shift(26)

## Leading Span B (senkou span B)
                      
### leading span b = $\frac{(52-PH)+(52-PL)}{2}$

PH - period high
PL - period low

In [10]:
data['senkou_span_B'] = ((data['High'].rolling(window=52).max() + data['Low'].rolling(window=52).min()) / 2).shift(26)

## Lagging Span (chikou span)

### lagging span = close plotted 26 periods in the past

In [11]:
data['chikou_span'] = data['Close'].shift(-26)

# Visualization
## Plotting all ichimoku and true data

In [12]:
func.plot_ichimoku(data)

### If price is above the cloud, and if conversion (tenkan sen) line and base (kijun sen) line are crossing each other, we buy

### If price is below the cloud, and if conversion (tenkan sen) line and base (kijun sen) line are crossing each other, we sell


In [13]:
last_date_with_value = func.get_last_date(end)
last_date_with_value

'2020-08-18'

## Getting latest data to check

In [14]:
curr_close_price, curr_span_a, curr_span_b, curr_kijun, curr_tenkan = func.get_latest_data(data,last_date_with_value)

## Calling ichimoku cloud method to check shall bot buy or sell

In [15]:
func.ichimoku_cloud_buy_sell(name, curr_close_price, curr_span_a, curr_span_b, curr_kijun, curr_tenkan)

SEEEELLL!!!!
Sold at the price of: 41.7599983215332 $
